### Install MLRun

In [1]:
# !pip install mlrun

### Load Config

In [2]:
import yaml
with open("config.yaml") as f:
    config = yaml.safe_load(f)

### Setup MLRun Project

In [62]:
import os
from os import path, getenv
import nuclio
from mlrun import new_project, code_to_function, run_local, NewTask, mlconf, import_function, mount_v3io, new_function

project_name = config["project"]["name"]
project_path = path.abspath('project')
project = new_project(name=project_name, context=project_path)
artifact_path = path.abspath('pipeline')
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Project name: {project_name}\nProject path: {project_path}')
print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Project name: horovod-pipeline-template
Project path: /User/igz_repos/igz-platform-deployment/horovod/project
Artifacts path: /User/igz_repos/igz-platform-deployment/horovod/pipeline
MLRun DB path: http://mlrun-api:8080


### Import Functions

In [63]:
# Syntax for normal MLRun job
project.set_function(func='utils.py',
                     name='utils',
                     kind='job',
                     image='mlrun/mlrun')

# Slightly different syntax for MPIJob
HOROVOD_FILE = os.path.join(project_path, config['trainer']['script'])
project.set_function(new_function(name='trainer',
                                  kind='mpijob',
                                  command=HOROVOD_FILE))

# Import function from MLRun function marketplace
project.set_function('hub://tf2_serving', 'serving')

### Pipeline

In [64]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
import yaml

with open("config.yaml") as f:
    config = yaml.safe_load(f)

funcs = {}

# Configure function resources
def init_functions(functions: dict, project=None, secrets=None):
    # Mount V3IO data layer to pipeline components
    for f in functions.values():
        f.apply(mount_v3io())
       
    # Configuration for training function
    image = lambda gpu: 'mlrun/ml-models-gpu' if gpu else 'mlrun/ml-models' 
    functions['trainer'].spec.image = image(config['trainer']['resources']['use_gpu'])
    functions['trainer'].with_requests(cpu=config['trainer']['resources']['requests']['cpu'],
                                       mem=config['trainer']['resources']['requests']['mem'])
    functions['trainer'].with_limits(cpu=config['trainer']['resources']['limits']['cpu'],
                                     mem=config['trainer']['resources']['limits']['mem'])
    if config['trainer']['resources']['use_gpu']:
        functions['trainer'].gpus(1)
    
    # Configuration for serving function
    functions['serving'].set_env('MODEL_CLASS', config['serving']['model_class'])
    functions['serving'].set_env('IMAGE_HEIGHT', config['serving']['image_height'])
    functions['serving'].set_env('IMAGE_WIDTH', config['serving']['image_width'])
    functions['serving'].set_env('ENABLE_EXPLAINER', config['serving']['enable_explainer'])
    functions['serving'].spec.min_replicas = config['serving']['min_replicas']

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name='Image classification demo',
    description='Train an Image Classification TF Algorithm using MLRun'
)
def kfpipeline():

    # step 1: download images
    open_archive = funcs['utils'].as_step(name='download',
                                          handler='open_archive',
                                          params={'target_path': config['utils']['images_dir']},
                                          inputs={'archive_url': config['utils']['image_archive']},
                                          outputs=['content'])

    # step 2: label images
    source_dir = str(open_archive.outputs['content']) + '/cats_n_dogs'
    label = funcs['utils'].as_step(name='label',
                                   handler='categories_map_builder',
                                   params={'source_dir': source_dir},
                                   outputs=['categories_map',
                                            'file_categories'])

    # step 3: train the model
    params = config['trainer']['params']
    params['data_path'] = source_dir
    train = funcs['trainer'].as_step(name='train',
                                     params=params,
                                     inputs={
                                         'categories_map': label.outputs['categories_map'],
                                         'file_categories': label.outputs['file_categories']},
                                     outputs=['model'])

    # deploy the model using nuclio functions
    deploy = funcs['serving'].deploy_step(models={config['serving']['model_name']: train.outputs['model']})


Overwriting /User/igz_repos/igz-platform-deployment/horovod/project/workflow.py


### Save Pipeline

In [65]:
project.set_workflow('main', 'workflow.py')
project.save()

In [66]:
run_id = project.run(
    'main',
    arguments={}, 
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}')), 
    dirty=True)

> 2021-01-08 00:47:25,160 [info] Pipeline run id=a93d0b26-7619-4f92-9e3e-3825a7f44c3f, check UI or DB for progress


In [57]:
from mlrun import wait_for_pipeline_completion
wait_for_pipeline_completion(run_id, timeout=3600);

RuntimeError: run status Failed not in expected statuses

## Test the serving function

After the function has been deployed we can test it as a regular REST Endpoint using `requests`.

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

### Define test params

In [ ]:
# Testing event
cat_image_url = 'https://s3.amazonaws.com/iguazio-sample-data/images/catanddog/cat.102.jpg'
response = requests.get(cat_image_url)
cat_image = response.content
img = Image.open(BytesIO(cat_image))

print('Test image:')
plt.imshow(img)

### Test The Serving Function (with Image URL)

In [ ]:
addr = 'http://nuclio-{}-{}:8080'.format(hvdproj.name, hvdproj.func('serving').metadata.name)

headers = {'Content-type': 'image/jpeg'}
url = addr + f'/cat_vs_dog_tf{tf_ver}/predict'

response = requests.post(url=url, 
                         data=json.dumps({'data_url': cat_image_url}), 
                         headers=headers)
print(response.content.decode('utf-8'))

In [ ]:
%%timeit 
requests.post(url=url, 
              data=json.dumps({'data_url': cat_image_url}), 
              headers=headers)

### Test The Serving Function (with Jpeg Image)

In [ ]:
headers = {'Content-type': 'image/jpeg'}
response = requests.post(url=url, 
                         data=cat_image, 
                         headers=headers)
print(response.content.decode('utf-8'))

In [ ]:
%%timeit
requests.post(url=url, 
              data=cat_image, 
              headers=headers)

**[back to top](#top)**